In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from aequitas.flow.methods.postprocessing import BalancedGroupThreshold
from aequitas.audit import Audit
from ucimlrepo import fetch_ucirepo
import experiment_util as util
import models

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


# [Correct a Model's Predictions](https://colab.research.google.com/github/dssg/aequitas/blob/notebooks/aequitas_flow_model_audit_and_correct.ipynb)

In [2]:
raw_df=fetch_ucirepo(id=2).data.original
df=raw_df.copy()
df["income"]=df["income"].replace(['<=50K.','>50K.'],['<=50K','>50K'])
df=df.replace(['?'],[np.nan])
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
48838,64,NaN,321403,HS-grad,9,Widowed,NaN,Other-relative,Black,Male,0,0,40,United-States,<=50K
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [3]:
pre_df=util.preporcess_transform(df)
train,test=train_test_split(pre_df,test_size=0.2)
train,valid=train_test_split(train,test_size=0.375)
print(train.shape)
print(valid.shape)
print(test.shape)

(24420, 15)
(14653, 15)
(9769, 15)


In [4]:
model=RandomForestClassifier()
train_x,train_y=models.get_feautures(train,"income")
model.fit(train_x,train_y)

RandomForestClassifier()

In [5]:
valid_x,valid_y=models.get_feautures(valid,"income")
valid_p = pd.Series(model.predict(valid_x),index=valid_x.index)
test_x,test_y=models.get_feautures(test,"income")
test_p = pd.Series(model.predict(test_x),index=test_x.index)

In [6]:
performance_metric = "tpr"
fairness_metric = "fpr"

# The column name of the sensitive attribute can be obtained from the aequitas.flow.Dataset object
fairness_column = "sex"
# The reference group for the example of BAF is "0", i.e. individuals from the younger group (<50).
reference_group = "Male"

In [7]:
audit_df=df.loc[test.index,["sex"]]
audit_df["label_value"] = test_y
# These might need to change if you are not using an aequitas.flow.Dataset object

audit_df["score"] = test_p
audit_df

,sex,label_value,score
13897,Female,0,0
34166,Male,0,0
11708,Male,0,0
23777,Female,0,0
346,Male,0,0
...,...,...,...
20428,Male,0,0
23300,Female,0,0
34808,Male,0,1
34574,Female,0,0


In [8]:
audit = Audit(audit_df,label_column="label_value",score_column="score", reference_groups={fairness_column:reference_group})
audit.performance()[performance_metric]

0    0.616589
Name: tpr, dtype: float64

In [9]:
audit.audit()
audit.summary_plot(fairness_metric)

c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\aequitas\plot\summary_chart.py:373: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  .agg(
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\aequitas\plot\summary_chart.py:373: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  .agg(
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\aequitas\plot\summary_chart.py:373: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To

alt.VConcatChart(...)

In [10]:
audit.disparity_plot(fairness_metric, fairness_column)

c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\vegalite\v5\api.py:405: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\vegalite\v5\api.py:405: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\

alt.LayerChart(...)

In [11]:
audit.metrics_df[["attribute_name", "attribute_value", "tpr", "fpr", "accuracy", "precision"]]

,attribute_name,attribute_value,tpr,fpr,accuracy,precision
0,sex,Female,0.577657,0.019788,0.935415,0.785185
1,sex,Male,0.623747,0.097654,0.816412,0.740190


In [12]:
threshold = BalancedGroupThreshold(threshold_type="top_pct", threshold_value=0.01, fairness_metric="fpr")

threshold.fit(valid_x, valid_p, valid_y, valid_x["sex"])

[INFO] 2024-02-11 18:01:26 methods.postprocessing.Threshold - Instantiating postprocessing Threshold.


c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\aequitas\flow\methods\postprocessing\balanced_group_threshold.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  group_df["value"].fillna(method="ffill", inplace=True)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\aequitas\flow\methods\postprocessing\balanced_group_threshold.py:101: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.


[INFO] 2024-02-11 18:01:26 methods.postprocessing.Threshold - Instantiating postprocessing Threshold.


INFO:methods.postprocessing.Threshold:Instantiating postprocessing Threshold.


In [13]:
corrected_test_bin_p = threshold.transform(test_x, test_p, test_x["sex"])

[INFO] 2024-02-11 18:01:26 methods.postprocessing.Threshold - Transforming predictions.


INFO:methods.postprocessing.Threshold:Transforming predictions.


[INFO] 2024-02-11 18:01:26 methods.postprocessing.Threshold - Finished transforming predictions.


INFO:methods.postprocessing.Threshold:Finished transforming predictions.


[INFO] 2024-02-11 18:01:26 methods.postprocessing.Threshold - Transforming predictions.


INFO:methods.postprocessing.Threshold:Transforming predictions.


[INFO] 2024-02-11 18:01:26 methods.postprocessing.Threshold - Finished transforming predictions.


INFO:methods.postprocessing.Threshold:Finished transforming predictions.


In [14]:
audit_df_fixed=df.loc[test.index,["sex"]]
audit_df_fixed["label_value"] = test_y
# These might need to change if you are not using an aequitas.flow.Dataset object

audit_df_fixed["score"] = corrected_test_bin_p
audit_df_fixed

,sex,label_value,score
13897,Female,0,0
34166,Male,0,0
11708,Male,0,0
23777,Female,0,0
346,Male,0,0
...,...,...,...
20428,Male,0,0
23300,Female,0,0
34808,Male,0,1
34574,Female,0,0


In [15]:
audit_fixed = Audit(audit_df_fixed,label_column="label_value",score_column="score", reference_groups={fairness_column: reference_group})
audit_fixed.performance()[performance_metric]

0    0.616589
Name: tpr, dtype: float64

In [16]:
audit_fixed.audit()
audit_fixed.summary_plot(fairness_metric)

c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\aequitas\plot\summary_chart.py:373: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  .agg(
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\aequitas\plot\summary_chart.py:373: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  .agg(
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\aequitas\plot\summary_chart.py:373: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To

alt.VConcatChart(...)

In [17]:
audit.disparity_plot(fairness_metric, fairness_column)

c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\vegalite\v5\api.py:405: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\vegalite\v5\api.py:405: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\

alt.LayerChart(...)

In [18]:
audit_fixed.metrics_df[["attribute_name", "attribute_value", "tpr", "fpr", "accuracy", "precision"]]

,attribute_name,attribute_value,tpr,fpr,accuracy,precision
0,sex,Female,0.577657,0.019788,0.935415,0.785185
1,sex,Male,0.623747,0.097654,0.816412,0.740190


Nothing has changed with the postprocessing. It is unclear to me why the postprocessing change nothing of the predictions.